In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import trimesh
import argparse
import os
import time
import numpy as np
import torch
import torch.optim as optim
import lib.utils as utils
from lib.utils import standard_normal_logprob
from lib.utils import count_nfe, count_total_time
from lib.utils import build_model_tabular

from lib.visualize_flow import visualize_transform
import lib.layers.odefunc as odefunc
from matplotlib import pyplot as plt
%matplotlib inline 

In [2]:
SOLVERS = ["dopri5"]
parser = argparse.ArgumentParser('SoftFlow')
parser.add_argument(
    '--data', choices=['2spirals_1d','2spirals_2d', 'swissroll_1d','swissroll_2d', 'circles_1d', 'circles_2d', '2sines_1d', 'target_1d'],
    type=str, default='3d_microgel'
)
parser.add_argument("--layer_type", type=str, default="concatsquash", choices=["concatsquash"])
parser.add_argument('--dims', type=str, default='64-64-64')
parser.add_argument("--num_blocks", type=int, default=1, help='Number of stacked CNFs.')
parser.add_argument('--time_length', type=float, default=0.5)
parser.add_argument('--train_T', type=eval, default=True)
parser.add_argument("--divergence_fn", type=str, default="brute_force", choices=["brute_force", "approximate"])
parser.add_argument("--nonlinearity", type=str, default="tanh", choices=odefunc.NONLINEARITIES)

parser.add_argument('--solver', type=str, default='dopri5', choices=SOLVERS)
parser.add_argument('--atol', type=float, default=1e-5)
parser.add_argument('--rtol', type=float, default=1e-5)

parser.add_argument('--residual', type=eval, default=False, choices=[True, False])
parser.add_argument('--rademacher', type=eval, default=False, choices=[True, False])
parser.add_argument('--spectral_norm', type=eval, default=False, choices=[True, False])
parser.add_argument('--niters', type=int, default=36000)
parser.add_argument('--batch_size', type=int, default=100)
parser.add_argument('--test_batch_size', type=int, default=1000)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--weight_decay', type=float, default=1e-5)

# for the proposed method
parser.add_argument('--std_min', type=float, default=0.0)
parser.add_argument('--std_max', type=float, default=0.1)
parser.add_argument('--std_weight', type=float, default=2)

parser.add_argument('--viz_freq', type=int, default=100)
parser.add_argument('--val_freq', type=int, default=400)
parser.add_argument('--log_freq', type=int, default=10)
parser.add_argument('--gpu', type=int, default=0)
# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
# logger
save_path = './results/' + args.data + '/SoftFlow'
utils.makedirs(save_path)

device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [4]:
def get_transforms(model):

    def sample_fn(z, logpz=None):
        zeors_std = torch.zeros(z.shape[0], 1).to(z)
        if logpz is not None:
            return model(z, zeors_std, logpz, reverse=True)
        else:
            return model(z, zeors_std, reverse=True)

    def density_fn(x, logpx=None):
        zeors_std = torch.zeros(x.shape[0], 1).to(x)
        if logpx is not None:
            return model(x, zeors_std, logpx, reverse=False)
        else:
            return model(x, zeors_std, reverse=False)

    return sample_fn, density_fn

DATA = trimesh.load('ANDREY_FOCTS1.ply')[:]
 
def compute_loss(args, model, batch_size=None):
    if batch_size is None: batch_size = args.batch_size

    # load data
#     x = toy_data.inf_train_gen(args.data, batch_size=batch_size)
    x = DATA[:,0:3]


    x = torch.from_numpy(x).type(torch.float32).to(device)
    zero = torch.zeros(x.shape[0], 1).to(x)

    # transform to z
    std = (args.std_max - args.std_min) * torch.rand_like(x[:,0]).view(-1,1) + args.std_min
    eps = torch.randn_like(x) * std
    std_in = std / args.std_max * args.std_weight
#     print(x.shape, std_in.shape, zero.shape)
    z, delta_logp = model(x+eps, std_in, zero)

    # compute log q(z)
    logpz = standard_normal_logprob(z).sum(1, keepdim=True)

    logpx = logpz - delta_logp
    loss = -torch.mean(logpx)
    return loss

In [5]:
model = build_model_tabular(args, 3).to(device)

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

time_meter = utils.RunningAverageMeter(0.93)
loss_meter = utils.RunningAverageMeter(0.93)
nfef_meter = utils.RunningAverageMeter(0.93)
nfeb_meter = utils.RunningAverageMeter(0.93)
tt_meter = utils.RunningAverageMeter(0.93)

end = time.time()
best_loss = float('inf')

In [6]:
LOW = -4
HIGH = 4
def visualize3d(samples, prior_sample, prior_logdensity, n_samples, transform, inverse_transform, memory=100, title1="$x ~ p(x)$", title2="$q(x)$", title3="$x ~ q(x)$", device="cpu", npts = 100):
    ## first one
    fig = plt.figure(figsize=(25,5))
    ax1 = fig.add_subplot(1, 3, 1, projection="3d")
    ax1.scatter(samples[:, 0], samples[:, 1], samples[:, 2], s=1, c='b', marker="s", facecolor="red", lw=0, alpha=1)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(title1)
    
    ## second one
    side = np.linspace(LOW, HIGH, npts)
    xx, yy, zz = np.meshgrid(side, side, side)
    memory=100
    x = np.hstack([xx.reshape(-1, 1), yy.reshape(-1, 1), zz.reshape(-1, 1)])
    x = torch.from_numpy(x).type(torch.float32).to(device)
    zeros = torch.zeros(x.shape[0], 1).to(x)

    z, delta_logp = [], []
    inds = torch.arange(0, x.shape[0]).to(torch.int64)
    for ii in torch.split(inds, int(memory**2)):
        z_, delta_logp_ = inverse_transform(x[ii], zeros[ii])
        z.append(z_)
        delta_logp.append(delta_logp_)
    z = torch.cat(z, 0)
    delta_logp = torch.cat(delta_logp, 0)

    logpz = prior_logdensity(z).view(z.shape[0], -1).sum(1, keepdim=True)  # logp(z)
    logpx = logpz - delta_logp

    px = np.exp(logpx.detach().cpu().numpy()).reshape(npts, npts,npts)
    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(px[:,:,1])
    ax.set_title(title2)
    
    ## Third one
    z = prior_sample(n_samples, 3).type(torch.float32).to(device)
    zk = []
    inds = torch.arange(0, z.shape[0]).to(torch.int64)
    for ii in torch.split(inds, int(memory**2)):
        zk.append(transform(z[ii]))

    zk = torch.cat(zk, 0).detach().cpu().numpy()

    ax1 = fig.add_subplot(1, 3, 3, projection="3d")
    ax1.scatter(zk[:, 0], zk[:, 1], zk[:, 2], s=1, c='r', marker="s", facecolor="red", lw=0, alpha=1)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(title3)
    
# plt.clf()    
# p_samples = DATA[:,0:3]
# visualize3d(p_samples, torch.randn, standard_normal_logprob, n_samples, sample_fn,density_fn, device=device)
# # plt.show()
# fig_filename = os.path.join('./', 'figs', '{:04d}.jpg'.format(1))
# utils.makedirs(os.path.dirname(fig_filename))
# plt.savefig(fig_filename, format='png', dpi=1200)
# plt.close()

In [ ]:
model.train()
for itr in range(1, args.niters + 1):
    optimizer.zero_grad()

    loss = compute_loss(args, model)
    

    loss_meter.update(loss.item())

    total_time = count_total_time(model)
    nfe_forward = count_nfe(model)

    loss.backward()
    optimizer.step()

    nfe_total = count_nfe(model)
    nfe_backward = nfe_total - nfe_forward
    nfef_meter.update(nfe_forward)
    nfeb_meter.update(nfe_backward)

    time_meter.update(time.time() - end)
    tt_meter.update(total_time)

    log_message = (
        'Iter {:04d} | Time {:.4f}({:.4f}) | Loss {:.6f}({:.6f}) | NFE Forward {:.0f}({:.1f})'
        ' | NFE Backward {:.0f}({:.1f}) | CNF Time {:.4f}({:.4f})'.format(
            itr, time_meter.val, time_meter.avg, loss_meter.val, loss_meter.avg, nfef_meter.val, nfef_meter.avg,
            nfeb_meter.val, nfeb_meter.avg, tt_meter.val, tt_meter.avg
        )
    )

    print(log_message)
    
    if itr % args.val_freq == 0 or itr == args.niters:
        with torch.no_grad():
            model.eval()
            test_loss = compute_loss(args, model, batch_size=args.test_batch_size)
            test_nfe = count_nfe(model)
            log_message = '[TEST] Iter {:04d} | Test Loss {:.6f} | NFE {:.0f}'.format(itr, test_loss, test_nfe)
            print(log_message)

            if test_loss.item() < best_loss:
                best_loss = test_loss.item()
                utils.makedirs(save_path)
                torch.save({
                    'args': args,
                    'state_dict': model.state_dict(),
                }, os.path.join(save_path, 'checkpt.pth'))
            model.train()

    if itr % args.viz_freq == 0:
        with torch.no_grad():
            model.eval()
#             p_samples = toy_data.inf_train_gen(args.data, batch_size=2000)
            p_samples = DATA[:,0:3]

            sample_fn, density_fn = get_transforms(model)

            plt.figure(figsize=(9, 3))
            visualize3d(p_samples, torch.randn, standard_normal_logprob, p_samples.shape[0], sample_fn,density_fn, device=device)
            fig_filename = os.path.join(save_path, 'figs', '{:04d}.jpg'.format(itr))
            utils.makedirs(os.path.dirname(fig_filename))
            plt.savefig(fig_filename, format='png', dpi=1200)
            plt.close()
            model.train()

    end = time.time()

print('Training has finished.')

Iter 0001 | Time 88.0963(7.1036) | Loss -0.566695(0.131992) | NFE Forward 38(36.8) | NFE Backward 44(40.3) | CNF Time 0.5000(0.5000)
Iter 0002 | Time 1.0714(6.6814) | Loss -0.588127(0.081584) | NFE Forward 38(36.9) | NFE Backward 44(40.5) | CNF Time 0.5000(0.5000)
Iter 0003 | Time 1.0694(6.2886) | Loss -0.607913(0.033319) | NFE Forward 38(37.0) | NFE Backward 44(40.8) | CNF Time 0.5000(0.5000)
Iter 0004 | Time 1.0683(5.9231) | Loss -0.623924(-0.012688) | NFE Forward 38(37.1) | NFE Backward 44(41.0) | CNF Time 0.5000(0.5000)
Iter 0005 | Time 1.0689(5.5833) | Loss -0.635768(-0.056304) | NFE Forward 38(37.1) | NFE Backward 44(41.2) | CNF Time 0.5000(0.5000)
Iter 0006 | Time 1.0750(5.2678) | Loss -0.650307(-0.097884) | NFE Forward 38(37.2) | NFE Backward 44(41.4) | CNF Time 0.5000(0.5000)
Iter 0007 | Time 1.1773(4.9814) | Loss -0.656779(-0.137006) | NFE Forward 38(37.2) | NFE Backward 50(42.0) | CNF Time 0.5000(0.5000)
Iter 0008 | Time 1.1732(4.7149) | Loss -0.663374(-0.173852) | NFE Forwa

Iter 0063 | Time 1.0755(1.1406) | Loss -0.945792(-0.885305) | NFE Forward 38(38.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0064 | Time 1.0759(1.1361) | Loss -0.949931(-0.889829) | NFE Forward 38(38.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0065 | Time 1.0742(1.1318) | Loss -0.959375(-0.894697) | NFE Forward 38(38.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0066 | Time 1.0744(1.1278) | Loss -0.959736(-0.899250) | NFE Forward 38(38.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0067 | Time 1.0750(1.1241) | Loss -0.962586(-0.903684) | NFE Forward 38(38.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0068 | Time 1.0764(1.1207) | Loss -0.967235(-0.908132) | NFE Forward 38(38.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0069 | Time 1.0743(1.1175) | Loss -0.970916(-0.912527) | NFE Forward 38(38.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 0070 | Time 1.0736(1.1144) | Loss -0.970099(-0.916557) | NFE For

Iter 0125 | Time 1.3320(1.3181) | Loss -1.140777(-1.104668) | NFE Forward 44(43.8) | NFE Backward 56(55.3) | CNF Time 0.5000(0.5000)
Iter 0126 | Time 1.3347(1.3193) | Loss -1.148244(-1.107718) | NFE Forward 44(43.8) | NFE Backward 56(55.4) | CNF Time 0.5000(0.5000)
Iter 0127 | Time 1.3268(1.3198) | Loss -1.149400(-1.110636) | NFE Forward 44(43.9) | NFE Backward 56(55.4) | CNF Time 0.5000(0.5000)
Iter 0128 | Time 1.3244(1.3201) | Loss -1.147625(-1.113225) | NFE Forward 44(43.9) | NFE Backward 56(55.5) | CNF Time 0.5000(0.5000)
Iter 0129 | Time 1.3274(1.3206) | Loss -1.152082(-1.115945) | NFE Forward 44(43.9) | NFE Backward 56(55.5) | CNF Time 0.5000(0.5000)
Iter 0130 | Time 1.3444(1.3223) | Loss -1.158681(-1.118937) | NFE Forward 44(43.9) | NFE Backward 56(55.5) | CNF Time 0.5000(0.5000)
Iter 0131 | Time 1.3408(1.3236) | Loss -1.160707(-1.121861) | NFE Forward 44(43.9) | NFE Backward 56(55.6) | CNF Time 0.5000(0.5000)
Iter 0132 | Time 1.3328(1.3243) | Loss -1.161701(-1.124649) | NFE For

Iter 0187 | Time 1.3316(1.3533) | Loss -1.282629(-1.256765) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0188 | Time 1.3338(1.3519) | Loss -1.283933(-1.258667) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0189 | Time 1.3265(1.3502) | Loss -1.289182(-1.260803) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0190 | Time 1.3303(1.3488) | Loss -1.289520(-1.262813) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0191 | Time 1.3357(1.3478) | Loss -1.291216(-1.264801) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0192 | Time 1.3386(1.3472) | Loss -1.299143(-1.267205) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0193 | Time 1.3284(1.3459) | Loss -1.294411(-1.269110) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0194 | Time 1.3320(1.3449) | Loss -1.302255(-1.271430) | NFE For

Iter 0249 | Time 1.3273(1.3325) | Loss -1.373490(-1.358721) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0250 | Time 1.3326(1.3325) | Loss -1.373605(-1.359763) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0251 | Time 1.3331(1.3325) | Loss -1.377375(-1.360996) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0252 | Time 1.3329(1.3326) | Loss -1.373293(-1.361857) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0253 | Time 1.3325(1.3326) | Loss -1.383817(-1.363394) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0254 | Time 1.3322(1.3325) | Loss -1.377068(-1.364351) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0255 | Time 1.3341(1.3326) | Loss -1.374746(-1.365079) | NFE Forward 44(44.0) | NFE Backward 56(56.0) | CNF Time 0.5000(0.5000)
Iter 0256 | Time 1.3270(1.3323) | Loss -1.379850(-1.366113) | NFE For

...

Iter 5635 | Time 2.3080(2.2925) | Loss -1.499175(-1.501925) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5636 | Time 2.2870(2.2921) | Loss -1.504943(-1.502136) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5637 | Time 2.2869(2.2917) | Loss -1.495803(-1.501693) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5638 | Time 2.3600(2.2965) | Loss -1.499152(-1.501515) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5639 | Time 2.3436(2.2998) | Loss -1.506392(-1.501856) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5640 | Time 2.2762(2.2982) | Loss -1.502347(-1.501891) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5641 | Time 2.2889(2.2975) | Loss -1.499925(-1.501753) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5642 | Time 2.3056(2.2981) | Loss -1.499431(-1.501590) | NFE For

Iter 5697 | Time 2.2970(2.3244) | Loss -1.500767(-1.501207) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5698 | Time 2.5039(2.3370) | Loss -1.498705(-1.501032) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5699 | Time 2.2987(2.3343) | Loss -1.501417(-1.501059) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5700 | Time 2.2910(2.3312) | Loss -1.501766(-1.501108) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5701 | Time 2.3791(2.3346) | Loss -1.501549(-1.501139) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5702 | Time 2.3491(2.3356) | Loss -1.498812(-1.500976) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5703 | Time 2.3593(2.3373) | Loss -1.501034(-1.500980) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5704 | Time 2.3397(2.3374) | Loss -1.502294(-1.501072) | NFE For

Iter 5759 | Time 2.3705(2.3710) | Loss -1.504403(-1.501797) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5760 | Time 2.3668(2.3707) | Loss -1.502909(-1.501875) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5761 | Time 2.3807(2.3714) | Loss -1.500618(-1.501787) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5762 | Time 2.3618(2.3707) | Loss -1.500885(-1.501724) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5763 | Time 2.3821(2.3715) | Loss -1.503605(-1.501855) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5764 | Time 2.3758(2.3718) | Loss -1.502813(-1.501922) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5765 | Time 2.3581(2.3709) | Loss -1.503093(-1.502004) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5766 | Time 2.3740(2.3711) | Loss -1.505896(-1.502277) | NFE For

Iter 5821 | Time 2.3004(2.2959) | Loss -1.503159(-1.501899) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5822 | Time 2.3999(2.3032) | Loss -1.501769(-1.501890) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5823 | Time 2.4969(2.3167) | Loss -1.502459(-1.501930) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5824 | Time 2.3323(2.3178) | Loss -1.499392(-1.501752) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5825 | Time 2.3025(2.3167) | Loss -1.500584(-1.501671) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5826 | Time 2.2743(2.3138) | Loss -1.499307(-1.501505) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5827 | Time 2.2971(2.3126) | Loss -1.499251(-1.501347) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5828 | Time 2.2949(2.3114) | Loss -1.497463(-1.501076) | NFE For

Iter 5883 | Time 2.3645(2.3667) | Loss -1.494494(-1.500483) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5884 | Time 2.3669(2.3668) | Loss -1.500960(-1.500517) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5885 | Time 2.3673(2.3668) | Loss -1.500434(-1.500511) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5886 | Time 2.3864(2.3682) | Loss -1.501104(-1.500552) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5887 | Time 2.3703(2.3683) | Loss -1.508226(-1.501089) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5888 | Time 2.3662(2.3682) | Loss -1.500158(-1.501024) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5889 | Time 2.3597(2.3676) | Loss -1.505057(-1.501306) | NFE Forward 50(50.0) | NFE Backward 80(80.0) | CNF Time 0.5000(0.5000)
Iter 5890 | Time 2.3829(2.3687) | Loss -1.499408(-1.501174) | NFE For